In [1]:
#http imports
import requests
from bs4 import BeautifulSoup
from datetime import date
import csv
import os
import json
import urllib.request
import pandas as pd
from datetime import date
from datetime import datetime

# pass headers in a dict to the headers parameter to mimic a real human doing the request using a web browser
headers = {
    'accept': '*/*',
    'content-type': 'text/html;charset=UTF-8',
    'content-encoding': 'gzip',
    'origin': 'www.zillow.com',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36'
}

In [2]:
num_clothing = 300 #how many products to scrape
url = "https://us.kowtowclothing.com/collections/all"

#init details
brand = []
product_type = [] #name extracted from url (can be blank)
product_name = []
product_description = []
price = [] #price before sale (can be blank)
sales_price = [] #price customer pays
color = []
material = []
product_url = []
picture_url = []
timestamp = []
gender = [] #website did not have gender specified so all unisex
new_arrival = []

current_date = date.today()

#opening url
response = requests.get(url, headers=headers)

#reading content with BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

products_scraped = 0

product_link_tags = soup.find_all("a", {'class':'grid-product-link-wrapper'})
product_links = []
for tag in product_link_tags:
    product_links.append('https://us.kowtowclothing.com' + tag.get('href'))


In [3]:
page = 1
new_arrival_names = []
while page <= 100:
    #new arrivals
    arrivals_url = "https://us.kowtowclothing.com/collections/new-arrivals?page=" + str(page)
    new_arrivals = BeautifulSoup(requests.get(arrivals_url, headers=headers).content, 'html.parser')

    p_elements = new_arrivals.find_all('p')
    if p_elements[3].get_text() == "Sorry, there are no products in this collection":
        print('aaaaaaaaaaaaaaaaaaaaaaaaaaaaa')
        break

            

    new_arrival_links = new_arrivals.find_all("a", {"class":"grid-product-link-wrapper"})
    for tag in new_arrival_links:
        new_arrival_names.append(tag.get("href").split("/")[-1])
    print(page)
    page += 1
print(new_arrival_names)

    

    
    

1
2
3
4
aaaaaaaaaaaaaaaaaaaaaaaaaaaaa
['otti-jumper-indigo-check', 'composure-cardigan-primary-red', 'escape-crew-primary-red', 'triangle-skirt-indigo-denim', 'henri-crew-indigo', 'jules-shirt-math-grid', 'lotte-dress-math-grid', 'phoebe-dress-math-grid', 'sailor-jeans-earth-denim', 'triangle-skirt-earth-denim', 'workshop-jacket-earth-denim', 'kiki-top-red', 'kiki-dress-red', 'theory-dress-red', 'composure-cardigan-primary-blue', 'escape-crew-primary-blue', 'cable-sweater-marle', 'apron-dress-black', 'cable-sweater-leaf', 'henri-crew-leaf', 'henri-roll-neck-jumper-leaf', 'painter-jacket-workwear-blue-denim', 'fisherman-crew-indigo', 'henri-roll-neck-jumper-indigo', 'workshop-jacket-indigo-denim', 'standard-jeans-earth-denim', 'tool-tote-earth-denim', 'barrel-jeans-natural-denim', 'outline-shorts-natural-denim', 'sailor-jeans-natural-denim', 'tool-tote-natural-denim', 'theory-dress-watercolour-check', 'klee-dress-watercolour-check', 'goldie-top-watercolour-check', 'building-block-cap-sl

In [4]:
page_exists = True
page = 1

while page_exists and page <13:
    print("PAGE NUM", page)
    url = "https://us.kowtowclothing.com/collections/all?page=" + str(page)
    #opening url
    response = requests.get(url, headers=headers)

    #reading content with BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    products_scraped = 0
    
    #checking if page has any products
    p_elements = soup.find_all('p')
    if p_elements[3].get_text() == "Sorry, there are no products in this collection":
        break
    
    #getting products as links
    product_link_tags = soup.find_all("a", {'class':'grid-product-link-wrapper'})
    product_links = []
    for tag in product_link_tags:
        product_links.append('https://us.kowtowclothing.com' + tag.get('href'))
    
    #Check for new arrival
    for link in product_links:
        new = False
        for arrival_name in new_arrival_names:
            if arrival_name == link.split("/")[-1]:
                new = True
                break
        new_arrival.append(new)
        
    count = 0
    while products_scraped < num_clothing and count < len(product_links): #mouse pog
        product_page = BeautifulSoup(requests.get(product_links[count], headers=headers).content, 'html.parser')

        product_div = product_page.find('div', {'itemtype': 'http://schema.org/Product'}) 

        #urls, desc, and brand
        product_metas = product_div.find_all('meta')
        """
        0: product url
        1: description
        2: image url
        3: brand
        ...: not needed
        """
        product_url.append(product_metas[0].get('content'))
        product_description.append(product_metas[1].get('content'))
        picture_url.append(product_metas[2].get('content'))
        brand.append(product_metas[3].get('content'))


        #money and color
        product_misc = product_div.find_all('h2', {'class':'subtitle is-9'})
        product_misc_text = []
        for h2 in product_misc:
            product_misc_text.append(h2.get_text()) #0 and 1 price with \n, 3 is color
        #splitting price
        money_split = product_misc_text[0].split('\n') #0 is '', 1,2 are prices
        if len(product_misc_text) >= 3:
            price.append(money_split[1])
            sales_price.append(money_split[2])
            color.append(product_misc_text[2])
        else:
            price.append(money_split[0])
            sales_price.append('none')
            color.append('none')
        #product name
        name = product_div.find('h1', {'itemprop':'name'})
        product_name.append(name.get_text())

        #material
        material_div = product_page.find("div", {'class':"swatch js-swatch"})
        if material_div:
            data_tags = material_div['data-product-tags']
            data_tags_split = data_tags.split(",")
            material_info = []
            for tag in data_tags_split:
                if "fabric" in tag or "content" in tag or "process" in tag:
                    material_info.append(tag.split(":")[1][:-1])
            material.append(material_info)
        else:
            material.append("none")
        #gender and timestamp
        gender.append('unisex') #website does not have gendered clothing 
        timestamp.append(datetime.now())

        #product type
        product_type.append("none")
        
        count += 1
    page += 1

    

PAGE NUM 1
PAGE NUM 2
PAGE NUM 3
PAGE NUM 4
PAGE NUM 5
PAGE NUM 6
PAGE NUM 7
PAGE NUM 8
PAGE NUM 9
PAGE NUM 10
PAGE NUM 11
PAGE NUM 12
aaaaaaaaaaaaaaaaaaaaaaaaaaaaa


In [5]:
#csv output and dataframe
dataframe = pd.DataFrame(list(zip(brand, product_type, product_name, product_description, price, sales_price, color, material, product_url, picture_url, timestamp, gender, new_arrival)), columns = ["Brand", "Product Type", "Product Name", "Description", "Price", "Sales Price", "Color", "Material", "Product URL", "Picture URL", "Timestamp", "Gender", "New Arrival"])


In [6]:
dataframe


,Brand,Product Type,Product Name,Description,Price,Sales Price,Color,Material,Product URL,Picture URL,Timestamp,Gender,New Arrival
0,Kowtow,none,OTTI JUMPER,A high neck jumper featuring an all over check...,US$ 269,,INDIGO CHECK,"[single-jersey, fairganic, knitwear, seed-to-g...",https://us.kowtowclothing.com/products/otti-ju...,//cdn.shopify.com/s/files/1/2182/6091/products...,2022-03-15 21:46:14.278517,unisex,True
1,Kowtow,none,COMPOSURE CARDIGAN,A wrap cardigan with a relaxed fit and raglan ...,US$ 189,,PRIMARY RED,"[tuck-stitch, fairganic, knitwear, seed-to-gar...",https://us.kowtowclothing.com/products/composu...,//cdn.shopify.com/s/files/1/2182/6091/products...,2022-03-15 21:46:14.557241,unisex,True
2,Kowtow,none,ESCAPE CREW,A crew neck jumper made from organic cotton an...,US$ 189,,PRIMARY RED,"[seed-stitch, fairganic, knitwear, seed-to-gar...",https://us.kowtowclothing.com/products/escape-...,//cdn.shopify.com/s/files/1/2182/6091/products...,2022-03-15 21:46:14.816157,unisex,True
3,Kowtow,none,TRIANGLE SKIRT,An a-line panelled skirt featuring contrast to...,US$ 255,,INDIGO DENIM,"[12.5oz, fairganic, seed-to-garment]",https://us.kowtowclothing.com/products/triangl...,//cdn.shopify.com/s/files/1/2182/6091/products...,2022-03-15 21:46:15.094273,unisex,True
4,Kowtow,none,BALLOON JEANS,Mid-rise jeans with a curved balloon leg that ...,US$ 225,,INDIGO DENIM,"[12.5oz, fairganic, nickel-free, seed-to-garme...",https://us.kowtowclothing.com/products/balloon...,//cdn.shopify.com/s/files/1/2182/6091/products...,2022-03-15 21:46:15.378646,unisex,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,Kowtow,none,UNISEX STANDARD PANT,Unisex pants with a tapered leg and an elastic...,US$ 199,,BLACK,"[peached-canvas, fairganic, seed-to-garment]",https://us.kowtowclothing.com/products/standar...,//cdn.shopify.com/s/files/1/2182/6091/products...,2022-03-15 21:47:30.597837,unisex,False
211,Kowtow,none,UNISEX OXFORD SHIRT,"An oversized unisex shirt, made from lightweig...",US$ 189,,WHITE,"[lightweight-canvas, fairganic, seed-to-garment]",https://us.kowtowclothing.com/products/oxford-...,//cdn.shopify.com/s/files/1/2182/6091/products...,2022-03-15 21:47:30.986194,unisex,False
212,Kowtow,none,UNISEX SINGLE-USE PLANET TEE,"A classic fit, unisex t-shirt made from mid-we...",US$ 99,,WHITE,"[190gsm, fairganic, seed-to-garment]",https://us.kowtowclothing.com/products/buildin...,//cdn.shopify.com/s/files/1/2182/6091/products...,2022-03-15 21:47:31.259411,unisex,False
213,Kowtow,none,COMMON INTEREST READER,Built on the ethos that great minds think diff...,US$ 29,,ISSUE 01,[common-interest],https://us.kowtowclothing.com/products/common-...,//cdn.shopify.com/s/files/1/2182/6091/products...,2022-03-15 21:47:31.500561,unisex,False
